In [5]:
import pandas as pd
import numpy as np

URL = "https://docs.google.com/spreadsheets/d/1YwuNz9lKEx8zj3th5hHfI1Z7i2WKUGexfqPnrxn6jiw/export?format=csv"

RENAME_MAP = {
    "Horodateur": "Timestamp",
    "Pour vous, être productif, c'est avant tout...": "Definition_productivite",
    "En moyenne, combien d'heures dormez-vous par nuit?": "Sommeil_moyen",
    "Combien d'heures avez-vous dormi la nuit dernière?": "Sommeil_nuit_derniere",
    "À quelle fréquence avez-vous un sommeil réparateur ?": "Sommeil_reparateur",
    "Comment évalueriez-vous votre hygiène de vie actuelle ?": "Hygiene_vie",
    "À quelle fréquence pratiquez-vous une activité physique (sport, marche active, etc.) ?": "Frequence_sport",
    "En moyenne, combien de litres d'eau bois-tu par jour ?": "Eau_litres",
    "Consommation quotidienne de café ou boissons énergétiques (nombre de tasses/verres)": "Cafe",
    "Avez-vous l'impression d'avoir été efficace dans vos tâches aujourd'hui ?": "Efficacite_aujourdhui",
    "Quel est votre niveau de stress général ces derniers jours ?": "Stress",
    "Niveau moyen de productivité ces 7 derniers jours.": "Productivite_7j",
    "Niveau d\u2019énergie aujourd\u2019hui.": "Energie",
    "Quel est votre âge ?": "Age",
    "Quelle est votre situation actuelle ?": "Situation",
}

FREQ_MAP = {
    "Jamais": 0,
    "Parfois (1 à 2 fois par semaine)": 1,
    "3 à 4 fois par semaine": 2,
    "Souvent (3 à 5 fois par semaine)": 3,
    "Tous les jours": 4,
}

PROD_MAP = {
    "Mou du genou (Petite forme, beaucoup de distractions)": 1,
    "Propre (Efficacité correcte, boulot fait)": 2,
    "Déterminé (Très productif et concentré)": 3,
    "Speed (Pas mal de pression)": 2,
    "Que dalle (Rien fait du tout)": 0,
}

STRESS_MAP = {
    "Tranquille (Stress léger et gérable)": 1,
    "Mou du genou (Stress modéré)": 2,
    "Speed (Pas mal de pression)": 3,
    "Stress élevé / Très tendu": 4,
}

MOTS_CHIFFRES = {
    "zero": 0, "un": 1, "deux": 2, "trois": 3, "quatre": 4,
    "cinq": 5, "six": 6, "sept": 7, "huit": 8, "neuf": 9,
    "dix": 10, "onze": 11, "douze": 12,
}

BORNES = {
    "Sommeil_moyen":         (2.0, 12.0),
    "Sommeil_nuit_derniere": (2.0, 12.0),
    "Eau_litres":            (0.0,  5.0),
    "Cafe":                  (0.0, 10.0),
    "Stress":                (1.0,  5.0),
    "Energie":               (1.0,  5.0),
    "Productivite_7j":       (1.0,  5.0),
}


def _convertir_sommeil(val):
    if pd.isna(val):
        return np.nan
    val = str(val).lower()
    for mot, chiffre in MOTS_CHIFFRES.items():
        val = val.replace(mot, str(chiffre))
    val = (
        val
        .replace("heures", "").replace("heure", "")
        .replace("hres", "").replace("hrs", "")
        .replace("de temps", "").replace("de t", "")
        .replace("environ", "").replace("h", "")
        .replace("mnt", "").replace("mn", "")
        .strip()
    )
    val = val.replace("ou", "-").replace("à", "-").replace("a", "-").replace(" ", "")
    while "--" in val:
        val = val.replace("--", "-")
    val = val.strip("-")
    try:
        if "-" in val:
            parts = val.split("-")
            nums = [float(p) for p in parts if p.replace(".", "", 1).isdigit()]
            return np.mean(nums) if nums else np.nan
        return float(val) if val else np.nan
    except Exception:
        return np.nan


def _parse_mixed(val, text_map):
    if pd.isna(val):
        return np.nan
    try:
        return float(val)
    except (ValueError, TypeError):
        return text_map.get(str(val).strip(), np.nan)


def _nettoyer_eau(val):
    if pd.isna(val):
        return np.nan
    try:
        v = float(val)
        if v > 20:
            return v / 1000
        if v > 5:
            return v * 0.25
        return v
    except Exception:
        return np.nan


def _cap_colonne(series: pd.Series, min_val: float, max_val: float) -> pd.Series:
    median = series.clip(lower=min_val, upper=max_val).median()
    out_of_bounds = (series < min_val) | (series > max_val)
    series = series.clip(lower=min_val, upper=max_val)
    series[out_of_bounds] = median
    return series


def load_and_preprocess() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv(URL)

    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace("\u2019", "'", regex=False)
        .str.replace("\u2018", "'", regex=False)
    )

    # Renommage défensif par mots-clés pour les colonnes non mappées
    rename_fallback = {}
    for col in df.columns:
        col_lower = col.lower().strip()
        if col not in RENAME_MAP:
            if "nergie" in col_lower:
                rename_fallback[col] = "Energie"
            elif "productivit" in col_lower and "7" in col_lower:
                rename_fallback[col] = "Productivite_7j"
            elif "stress" in col_lower:
                rename_fallback[col] = "Stress"
            elif "situation" in col_lower:
                rename_fallback[col] = "Situation"
            elif "ge" in col_lower and len(col) < 20:
                rename_fallback[col] = "Age"

    df = df.rename(columns=RENAME_MAP)
    df = df.rename(columns=rename_fallback)

    df["Sommeil_moyen"]         = df["Sommeil_moyen"].apply(_convertir_sommeil)
    df["Sommeil_nuit_derniere"] = df["Sommeil_nuit_derniere"].apply(_convertir_sommeil)
    df["Frequence_sport"]       = df["Frequence_sport"].map(FREQ_MAP)
    df["Efficacite_aujourdhui"] = df["Efficacite_aujourdhui"].map(PROD_MAP)
    df["Productivite_7j"]       = pd.to_numeric(df["Productivite_7j"], errors="coerce")
    df["Stress"]                = df["Stress"].apply(_parse_mixed, text_map=STRESS_MAP)
    df["Energie"]               = pd.to_numeric(df["Energie"], errors="coerce")
    df["Cafe"]                  = pd.to_numeric(df["Cafe"], errors="coerce")
    df["Eau_litres"]            = df["Eau_litres"].apply(_nettoyer_eau)

    for col, (min_val, max_val) in BORNES.items():
        df[col] = _cap_colonne(df[col], min_val, max_val)

    num_cols = df.select_dtypes(include=np.number).columns
    df[num_cols] = df[num_cols].apply(
        lambda col: col.fillna(col.mode()[0] if not col.mode().empty else 0)
    )

    df_normalized = df.copy()
    df_normalized[num_cols] = df_normalized[num_cols].apply(
        lambda col: (col - col.min()) / (col.max() - col.min())
        if col.max() != col.min() else col
    )

    corr = df_normalized[num_cols].corr()

    return df, df_normalized, corr


df, df_normalized, corr = load_and_preprocess()

df

,Timestamp,Definition_productivite,Sommeil_moyen,Sommeil_nuit_derniere,Sommeil_reparateur,Hygiene_vie,Frequence_sport,Eau_litres,Cafe,Efficacite_aujourdhui,Stress,Productivite_7j,Energie,Age,Situation
0,25/02/2026 12:28:28,Travailler vite et efficacement,6.0,6.0,Parfois (1 à 2 fois par semaine),Moyenne,2.0,2.0,1,1,3.0,3,4,18-25 ans,Étudiant(e)
1,25/02/2026 12:29:20,Accomplir toutes ses tâches de la journée,6.0,6.0,Parfois (1 à 2 fois par semaine),Très bonne,4.0,3.0,1,2,3.0,2,3,18-25 ans,Étudiant(e)
2,25/02/2026 12:34:21,Travailler vite et efficacement,6.0,6.0,Souvent (3 à 5 fois par semaine),Très bonne,4.0,2.0,0,2,3.0,3,3,18-25 ans,Étudiant(e)
3,25/02/2026 12:35:16,Avoir l'esprit calme et concentré,6.0,6.0,Parfois (1 à 2 fois par semaine),Moyenne,4.0,3.0,1,2,4.0,3,3,18-25 ans,Étudiant(e)
4,25/02/2026 12:39:39,Travailler vite et efficacement,6.0,6.0,Souvent (3 à 5 fois par semaine),Moyenne,0.0,3.0,0,1,2.0,3,3,18-25 ans,Travailleur / Travailleuse
5,25/02/2026 12:42:39,Accomplir toutes ses tâches de la journée,6.0,6.0,Parfois (1 à 2 fois par semaine),Très bonne,4.0,2.0,0,1,4.0,2,2,18-25 ans,Étudiant(e)
6,25/02/2026 12:43:06,Travailler vite et efficacement,6.0,6.0,Parfois (1 à 2 fois par semaine),Pas terrible,4.0,1.0,0,2,1.0,3,3,18-25 ans,Étudiant(e)
7,25/02/2026 12:51:32,Travailler vite et efficacement,6.0,5.0,Jamais,Très bonne,2.0,1.0,1,2,1.0,3,4,18-25 ans,Étudiant(e)
8,25/02/2026 13:09:21,Travailler vite et efficacement,5.5,6.0,Parfois (1 à 2 fois par semaine),Moyenne,4.0,1.0,2,2,2.0,3,4,18-25 ans,Les deux
9,25/02/2026 13:09:57,Avoir l'esprit calme et concentré,5.5,5.0,Jamais,Moyenne,2.0,2.0,1,1,4.0,2,2,18-25 ans,Travailleur / Travailleuse


In [4]:
import pandas as pd
import numpy as np

FREQ_MAP = {
    "Jamais": 0,
    "Parfois (1 à 2 fois par semaine)": 1,
    "3 à 4 fois par semaine": 2,
    "Souvent (3 à 5 fois par semaine)": 3,
    "Tous les jours": 4,
}

PROD_MAP = {
    "Mou du genou (Petite forme, beaucoup de distractions)": 1,
    "Propre (Efficacité correcte, boulot fait)": 2,
    "Déterminé (Très productif et concentré)": 3,
    "Speed (Pas mal de pression)": 2,
    "Que dalle (Rien fait du tout)": 0,
}

STRESS_MAP = {
    "Tranquille (Stress léger et gérable)": 1,
    "Mou du genou (Stress modéré)": 2,
    "Speed (Pas mal de pression)": 3,
    "Stress élevé / Très tendu": 4,
}

MOTS_CHIFFRES = {
    "zero": 0, "un": 1, "deux": 2, "trois": 3, "quatre": 4,
    "cinq": 5, "six": 6, "sept": 7, "huit": 8, "neuf": 9,
    "dix": 10, "onze": 11, "douze": 12,
}

BORNES = {
    "Sommeil_moyen":         (2.0, 12.0),
    "Sommeil_nuit_derniere": (2.0, 12.0),
    "Eau_litres":            (0.0,  5.0),
    "Cafe":                  (0.0, 10.0),
    "Stress":                (1.0,  5.0),
    "Energie":               (1.0,  5.0),
    "Productivite_7j":       (1.0,  5.0),
}

RENAME_MAP = {
    "Horodateur": "Timestamp",
    "Pour vous, être productif, c'est avant tout...": "Definition_productivite",
    "En moyenne, combien d'heures dormez-vous par nuit?": "Sommeil_moyen",
    "Combien d'heures avez-vous dormi la nuit dernière?": "Sommeil_nuit_derniere",
    "À quelle fréquence avez-vous un sommeil réparateur ?": "Sommeil_reparateur",
    "Comment évalueriez-vous votre hygiène de vie actuelle ?": "Hygiene_vie",
    "À quelle fréquence pratiquez-vous une activité physique (sport, marche active, etc.) ?": "Frequence_sport",
    "En moyenne, combien de litres d'eau bois-tu par jour ?": "Eau_litres",
    "Consommation quotidienne de café ou boissons énergétiques (nombre de tasses/verres)": "Cafe",
    "Avez-vous l'impression d'avoir été efficace dans vos tâches aujourd'hui ?": "Efficacite_aujourdhui",
    "Quel est votre niveau de stress général ces derniers jours ?": "Stress",
    "Niveau moyen de productivité ces 7 derniers jours.": "Productivite_7j",
    "Niveau d\u2019énergie aujourd\u2019hui.": "Energie",
    "Quel est votre âge ?": "Age",
    "Quelle est votre situation actuelle ?": "Situation",
}

URL = "https://docs.google.com/spreadsheets/d/1YwuNz9lKEx8zj3th5hHfI1Z7i2WKUGexfqPnrxn6jiw/export?format=csv"


def _convertir_sommeil(val):
    if pd.isna(val):
        return np.nan
    val = str(val).lower()
    for mot, chiffre in MOTS_CHIFFRES.items():
        val = val.replace(mot, str(chiffre))
    val = (
        val
        .replace("heures", "").replace("heure", "")
        .replace("hres", "").replace("hrs", "")
        .replace("de temps", "").replace("de t", "")
        .replace("environ", "").replace("h", "")
        .replace("mnt", "").replace("mn", "")
        .strip()
    )
    val = (
        val
        .replace("ou", "-").replace("à", "-")
        .replace("a", "-").replace(" ", "")
    )
    while "--" in val:
        val = val.replace("--", "-")
    val = val.strip("-")
    try:
        if "-" in val:
            parts = val.split("-")
            nums = [float(p) for p in parts if p.replace(".", "", 1).isdigit()]
            return np.mean(nums) if nums else np.nan
        return float(val) if val else np.nan
    except Exception:
        return np.nan


def _parse_mixed(val, text_map):
    if pd.isna(val):
        return np.nan
    try:
        return float(val)
    except (ValueError, TypeError):
        return text_map.get(str(val).strip(), np.nan)


def _nettoyer_eau(val):
    if pd.isna(val):
        return np.nan
    try:
        v = float(val)
        if v > 20:
            return v / 1000
        if v > 5:
            return v * 0.25
        return v
    except Exception:
        return np.nan


def _cap_colonne(series: pd.Series, min_val: float, max_val: float) -> pd.Series:
    median = series.clip(lower=min_val, upper=max_val).median()
    out_of_bounds = (series < min_val) | (series > max_val)
    series = series.clip(lower=min_val, upper=max_val)
    series[out_of_bounds] = median
    return series


def load_and_preprocess() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv(URL)
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace("\u2019", "'", regex=False)
        .str.replace("\u2018", "'", regex=False)
    )
    df = df.rename(columns=RENAME_MAP)

    df["Sommeil_moyen"]         = df["Sommeil_moyen"].apply(_convertir_sommeil)
    df["Sommeil_nuit_derniere"] = df["Sommeil_nuit_derniere"].apply(_convertir_sommeil)
    df["Frequence_sport"]       = df["Frequence_sport"].map(FREQ_MAP)
    df["Efficacite_aujourdhui"] = df["Efficacite_aujourdhui"].map(PROD_MAP)
    df["Productivite_7j"]       = pd.to_numeric(df["Productivite_7j"], errors="coerce")
    df["Stress"]                = df["Stress"].apply(_parse_mixed, text_map=STRESS_MAP)
    df["Energie"]               = pd.to_numeric(df["Energie"], errors="coerce")
    df["Cafe"]                  = pd.to_numeric(df["Cafe"], errors="coerce")
    df["Eau_litres"]            = df["Eau_litres"].apply(_nettoyer_eau)

    for col, (min_val, max_val) in BORNES.items():
        df[col] = _cap_colonne(df[col], min_val, max_val)

    num_cols = df.select_dtypes(include=np.number).columns
    df[num_cols] = df[num_cols].apply(
        lambda col: col.fillna(col.mode()[0] if not col.mode().empty else 0)
    )

    df_normalized = df.copy()
    df_normalized[num_cols] = df_normalized[num_cols].apply(
        lambda col: (col - col.min()) / (col.max() - col.min())
        if col.max() != col.min() else col
    )

    corr = df_normalized[num_cols].corr()

    return df, df_normalized, corr


df, df_normalized, corr = load_and_preprocess()

df.head(10)

KeyError: 'Energie'